In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

import pandas as pd


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            print(f"loss: {loss:>7f}")


In [5]:
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return pred.argmax(1).numpy(), y.numpy()


In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = MNIST(root = './', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = MNIST(root = './', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)


0.3%

100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw


100.0%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw




29.8%

100.0%


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



100.0%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



In [7]:
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}")
    train_loop(train_dataloader, model, loss_fn, optimizer)


Epoch 1
loss: 2.301182
loss: 2.287291
loss: 2.262454
loss: 2.253855
loss: 2.203875
loss: 2.214497
loss: 2.162686
loss: 2.216417
loss: 2.162077
loss: 2.069524
Epoch 2
loss: 2.042907
loss: 2.052378
loss: 2.054356
loss: 2.078829
loss: 1.972134
loss: 1.924995
loss: 1.952487
loss: 1.838108
loss: 1.853783
loss: 1.837213
Epoch 3
loss: 1.801227
loss: 1.806959
loss: 1.638476
loss: 1.627393
loss: 1.647990
loss: 1.602678
loss: 1.554525
loss: 1.682244
loss: 1.577808
loss: 1.311918
Epoch 4
loss: 1.538504
loss: 1.445994
loss: 1.518119
loss: 1.205961
loss: 1.311170
loss: 1.295786
loss: 1.231373
loss: 1.302932
loss: 1.257061
loss: 1.327452
Epoch 5
loss: 1.499521
loss: 1.464302
loss: 1.102972
loss: 1.353311
loss: 1.101503
loss: 1.173289
loss: 0.869145
loss: 1.214229
loss: 1.179010
loss: 1.135705


In [8]:
predictions, labels = test_loop(test_dataloader, model)


Test Error: 
 Accuracy: 68.8%, Avg loss: 0.016962 



In [9]:
df = pd.DataFrame({'Predictions': predictions, 'Labels': labels})
df.to_csv('results.csv', index=False)
